In [38]:
import os
print(os.getcwd())
os.chdir("d:/vscode_machineLearning/BEST_PROJECTS/ForbsBillionaires")
print(os.getcwd())

d:\vscode_machineLearning\BEST_PROJECTS\ForbsBillionaires
d:\vscode_machineLearning\BEST_PROJECTS\ForbsBillionaires


## Entity

In [39]:
from pathlib import Path
from dataclasses import dataclass

In [40]:
@dataclass(frozen=True)
class DataValidationConfig:
    root_dir:Path
    STATUS_file :str
    data_dir:Path
    schema_check:dict

## configuration manager

In [41]:
from predictor.constatns import *
from predictor.utils import *

In [42]:
CONFIG_FILE_PATH

WindowsPath('config/config.yaml')

In [55]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):


        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        creat_dirs([self.config.artifacts_root])

    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.INDEPENDENT_FEATURES

        creat_dirs([config.root_dir])
        

        data_validation_config =  DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_file=config.STATUS_FILE,
            data_dir=config.data_dir,
            schema_check=schema
        )

        return data_validation_config
        
        
        

In [56]:
read_yaml(CONFIG_FILE_PATH)

[2023-09-13 11:37:29,988: INFO: utils: yaml file: config\config.yaml loaded successfully]


ConfigBox({'artifacts_root': 'artifacts', 'data_validation': {'root_dir': 'artifacts\\validation', 'data_dir': 'artifacts\\cleand_data\\billionaires.csv', 'STATUS_FILE': 'artifacts\\cleand_data\\status.txt'}})

In [57]:
read_yaml(SCHEMA_FILE_PATH)

[2023-09-13 11:37:30,270: INFO: utils: yaml file: schema.yaml loaded successfully]


ConfigBox({'INDEPENDENT_FEATURES': {'rank': 'int64', 'full_name': 'object', 'age': 'int64', 'gender': 'object', 'country_of_citizenship': 'object', 'city_of_residence': 'object', 'business_category': 'object', 'self_made': 'bool', 'wealth_status': 'object'}, 'DEPENDENT_FEATURES': {'net_worth': 'int64'}})

In [58]:
read_yaml(PARAMS_FILE_PATH)

[2023-09-13 11:37:30,531: INFO: utils: yaml file: params.yaml loaded successfully]


ConfigBox({'key': 'val'})

## components

In [59]:
from predictor import logger
import pandas as pd

In [60]:
config = read_yaml(CONFIG_FILE_PATH)

[2023-09-13 11:37:36,316: INFO: utils: yaml file: config\config.yaml loaded successfully]


In [61]:
type(config)

box.config_box.ConfigBox

In [62]:
config.artifacts_root

'artifacts'

In [63]:
config.data_validation.data_dir

'artifacts\\cleand_data\\billionaires.csv'

In [64]:
pd.read_csv(config.data_validation.data_dir)

,year,month,rank,net_worth,full_name,age,gender,country_of_citizenship,city_of_residence,business_category,self_made,wealth_status
0,2017,3,1,86.0,Bill Gates,61.0,Male,United States,Medina,Technology,True,Increased
1,2017,3,2,75.6,Warren Buffett,86.0,Male,United States,Omaha,Finance and Investments,True,Increased
2,2017,3,3,72.8,Jeff Bezos,53.0,Male,United States,Seattle,Technology,True,Increased
3,2017,3,4,71.3,Amancio Ortega,80.0,Male,Spain,La Coruna,Fashion & Retail,True,Increased
4,2017,3,5,56.0,Mark Zuckerberg,32.0,Male,United States,Palo Alto,Technology,True,Increased
...,...,...,...,...,...,...,...,...,...,...,...,...
16557,2023,4,2540,1.0,Yu Rong,51.0,Male,China,Shanghai,Healthcare,True,Decreased
16558,2023,4,2540,1.0,"Richard Yuengling, Jr.",80.0,Male,United States,Pottsville,Food & Beverage,False,Remained Even
16559,2023,4,2540,1.0,Zhang Gongyun,60.0,Male,China,Gaomi,Manufacturing,True,Returned to List
16560,2023,4,2540,1.0,Zhang Guiping & family,71.0,Male,China,Nanjing,Real Estate,True,Decreased


In [65]:
class DataValidataion:
    def __init__(self,config_in :DataValidationConfig):
        self.config_in = config_in
    
    def validate_features(self) -> bool:
        try:
            validation_status = False
            df = pd.read_csv(self.config_in.data_dir)
            all_features = list(df.columns)
            all_schema = self.config_in.schema_check

            for cols in all_features:
                if cols not in all_schema:
                    validation_status = False
                    with open(self.config_in.STATUS_file,'w') as stat_file:
                        stat_file.write(f"Validation status: {validation_status}")
                
                else:
                    validation_status = True
                    with open(self.config_in.STATUS_file,'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        except Exception as e:
            raise CustomException(e)
        

## pipeline

In [68]:
try:
    confg = ConfigurationManager()
    data_val_confg = config.get_data_validation_config()
    data_validation = DataValidataion(config_in=data_val_confg)
    data_validation.validate_features()

except Exception as e:
    raise CustomException(e)


# try:
#     config = ConfigurationManager()
#     data_val_config = config.get_data_validation_config()
#     data_validation = DataValidation(config_in=data_val_config)
#     data_validation.validate_features()
# except Exception as e:
#     raise CustomException(e,sys)


[2023-09-13 11:38:23,576: INFO: utils: yaml file: config\config.yaml loaded successfully]
[2023-09-13 11:38:23,579: INFO: utils: yaml file: params.yaml loaded successfully]
[2023-09-13 11:38:23,584: INFO: utils: yaml file: schema.yaml loaded successfully]
[2023-09-13 11:38:23,587: INFO: utils: Created artifacts]
[2023-09-13 11:38:23,588: INFO: utils: Created artifacts\validation]


# **The `params.yaml` was Empty that's why I was getting error.**